In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install wordcloud

In [ ]:
# import os
import re
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import random
import nltk
from sklearn.decomposition import PCA
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cluster import KMeans,MiniBatchKMeans
from sklearn.decomposition import PCA
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline
stopwords.words("english")[:10] # <-- import the english stopwords

###Load Data

In [ ]:
raw_data = pd.read_csv('/content/drive/MyDrive/Amazon_Product_data/dataset.csv')
raw_data.head()

In [ ]:
raw_data.shape

Using NLTK Pre-trained Sentiment Analyzer for getting the sentiment for the news headline.


In [ ]:

count = 0
for i,row in raw_data.iterrows():
  count+=1
  text = row['Headline']
  sia_object = SIA()
  sentiment_scores = sia_object.polarity_scores(text = text)
  print(sentiment_scores)
  max_sentiment = max(sentiment_scores, key= lambda x: sentiment_scores[x])
  # raw_data['sentiment']=max_sentiment
  print(max_sentiment)
print(count)


After doing nltk sentiment intensity analyzer ,we got only neutral sentiment over 50 news headlines.So we are not doing sentiment  prediction with supervised approach. We are focusing on the unsupervised approach.

#Clustering

###Text Pre-Processing

In [ ]:
raw_data.isnull().sum()

In [ ]:
def preprocess_text(text: str, remove_stopwords: bool) -> str:
    """This utility function sanitizes a string by:
    - removing links
    - removing special characters
    - removing numbers
    - removing stopwords
    - transforming in lowercase
    - removing excessive whitespaces
    Args:
        text (str): the input text you want to clean
        remove_stopwords (bool): whether or not to remove stopwords
    Returns:
        str: the cleaned text
    """

    # remove links
    text = re.sub(r"http\S+", "", text)
    # remove special chars and numbers
    text = re.sub("[^A-Za-z]+", " ", text)
    # remove stopwords
    if remove_stopwords:
        # 1. tokenize
        tokens = nltk.word_tokenize(text)
        # 2. check if stopword
        tokens = [w for w in tokens if not w.lower() in stopwords.words("english")]
        # 3. join back together
        text = " ".join(tokens)
    # return text in lower case and stripped of whitespaces
    text = text.lower().strip()
    # print(text)
    return text

In [ ]:
data1 = raw_data.copy()

In [ ]:
data2 = raw_data.copy()

In [ ]:
data1.head()

In [ ]:
data1.shape

In [ ]:
data2.shape

###Tf-Idf Vectorization

In [ ]:
def tf_idf(df):
  # initialize the vectorizer
  vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=5, max_df=0.95)
  # fit_transform applies TF-IDF to clean texts - we save the array of vectors in X
  X = vectorizer.fit_transform(df['cleaned_head_line'])
  print(X.shape)
  return X,vectorizer

###PCA

In [ ]:

def find_pca(feature_vector):
    # initialize PCA with 2 components
    pca = PCA(n_components=2, random_state=42)
    # pass our X to the pca and store the reduced vectors into pca_vecs
    pca_vecs = pca.fit_transform(feature_vector.toarray())
    # save our two dimensions into x0 and x1
    x0 = pca_vecs[:, 0]
    x1 = pca_vecs[:, 1]
    return x0,x1

### Elbow Method

In [ ]:

def elbow_method(pca_vector,method ='Kmeans'):
    """
    This is the function used to get optimal number of clusters in order to feed to the k-means clustering algorithm.
    """
    if method =='Kmeans':

      number_clusters = range(1, 7)  # Range of possible clusters that can be generated
      kmeans = [KMeans(n_clusters=i, max_iter = 600) for i in number_clusters] # Getting no. of clusters 

      score = [kmeans[i].fit(pca_vector).score(pca_vector) for i in range(len(kmeans))] # Getting score corresponding to each cluster.
      score = [i*-1 for i in score] # Getting list of positive scores.
      
      plt.plot(number_clusters, score)
      plt.xlabel('Number of Clusters')
      plt.ylabel('Score')
      plt.title('Elbow Method')
      plt.show()
    else:
      number_clusters = range(1, 7)  # Range of possible clusters that can be generated
      kmeans = [MiniBatchKMeans(n_clusters=i, max_iter = 600) for i in number_clusters] # Getting no. of clusters 

      score = [kmeans[i].fit(pca_vector).score(pca_vector) for i in range(len(kmeans))] # Getting score corresponding to each cluster.
      score = [i*-1 for i in score] # Getting list of positive scores.
      
      plt.plot(number_clusters, score)
      plt.xlabel('Number of Clusters')
      plt.ylabel('Score')
      plt.title('Elbow Method')
      plt.show()

###Keyword Extraction

In [ ]:
def get_top_keywords(clusters,n_terms,feature_vector,vectorizer):
    """This function returns the keywords for each centroid of the KMeans"""
    df = pd.DataFrame(feature_vector.todense()).groupby(clusters).mean() # groups the TF-IDF vector by cluster
    terms = vectorizer.get_feature_names_out() # access tf-idf terms
    for i,r in df.iterrows():
        print('\nCluster {}'.format(i))
        print(','.join([terms[t] for t in np.argsort(r)[-n_terms:]])) # for each row of the dataframe, find the n terms that have the highest tf idf score
            

In [ ]:
def show_pca(df):
    # set image size
  plt.figure(figsize=(12, 7))
  # set a title
  plt.title("TF-IDF + KMeans clustering", fontdict={"fontsize": 18})
  # set axes names
  plt.xlabel("X0", fontdict={"fontsize": 16})
  plt.ylabel("X1", fontdict={"fontsize": 16})
  # create scatter plot with seaborn, where hue is the class used to group the data
  sns.scatterplot(data=df, x='x0', y='x1', hue='Cluster', palette="viridis")
  plt.show()


In [ ]:
def show_wordcloud(text):

  print(text)
  # wordcloud = WordCloud().generate(text)
  wordcloud = WordCloud(width = 2000, height = 2000,
                  background_color ='white',
                  min_font_size = 10).generate(text)

  # plot the WordCloud image                      
  plt.figure(figsize = (8, 8), facecolor = None)
  plt.imshow(wordcloud)
  plt.axis("off")
  plt.tight_layout(pad = 0)
  
  # plt.show()
  # plt.imshow(wordcloud, interpolation='bilinear')
  # plt.axis("off")


K-Means Clustering

In [ ]:
def Kmeans_Clustering(df,method='Kmeans'):

  df['cleaned_head_line'] = df['Headline'].apply(lambda x: preprocess_text(x, remove_stopwords=True))
  feature_vector,vectorizer = tf_idf(df)
  x0,x1 = find_pca(feature_vector)
  df['x0'] = x0
  df['x1'] = x1
  elbow_method(feature_vector,method = method)
  # initialize kmeans with 3 centroids
  kmeans = KMeans(n_clusters=3, random_state=42)
  # fit the model
  kmeans.fit(feature_vector)
  # store cluster labels in a variable
  clusters = kmeans.labels_
  print([c for c in clusters][:10])
  df['Cluster'] = clusters
  print(Counter(kmeans.labels_))
  
  get_top_keywords(clusters,10,feature_vector,vectorizer)
  # map clusters to appropriate labels 
  cluster_map = {0: "Neutral", 1: "Positive", 2: "Negative"}
  # apply mapping
  df['Cluster'] = df['Cluster'].map(cluster_map)
  print(df.Cluster.value_counts())
  #Positive Word Cloud
  df_positive = df[df["Cluster"] == 'Positive']
  text_pos = df_positive.cleaned_head_line.str.cat()
  #Positive Word Cloud
  df_negative = df[df["Cluster"] == 'Negative']
  text_neg = df_negative.cleaned_head_line.str.cat()
  #Positive Word Cloud
  df_neutral = df[df["Cluster"] == 'Neutral']
  text_neu = df_neutral.cleaned_head_line.str.cat()

  print(df_positive.head())
  print(df_positive.shape)
  
  show_pca(df)
  show_wordcloud(text_pos)
  show_wordcloud(text_neg)
  show_wordcloud(text_neu)
  df.to_csv('Kmeans_Cluster_result.csv',index=False)
  return df

###BagOfWords

In [ ]:
#BOW
def bow_features(df):
    
    """
    Returns:
        _type_: _description_
    """
    vectorizer = CountVectorizer()
    x = vectorizer.fit_transform(df['cleaned_head_line'].tolist())            
    bow = x.toarray()   
    print(x.shape)
    
    return x,vectorizer

###MiniBatchKmeans

In [ ]:
def MBK_Clustering(df,method='MBK'):
  df['cleaned_head_line'] = df['Headline'].apply(lambda x: preprocess_text(x, remove_stopwords=True))
  feature_vector,vectorizer = bow_features(df)
  x0,x1 = find_pca(feature_vector)
  df['x0'] = x0
  df['x1'] = x1
  elbow_method(feature_vector,method = method)
  # initialize kmeans with 3 centroids
  mbk = MiniBatchKMeans(init ='k-means++', n_clusters = 3,
                      batch_size = 10, n_init = 10,
                      max_no_improvement = 10, verbose = 0)

  mbk.fit(feature_vector)
  clusters = mbk.labels_
  print([c for c in clusters][:10])
  df['Cluster'] = clusters
  print(Counter(mbk.labels_))
  
  get_top_keywords(clusters,10,feature_vector,vectorizer)
  # map clusters to appropriate labels 
  cluster_map = {0: "Neutral", 1: "Positive", 2: "Negative"}
  # apply mapping
  df['Cluster'] = df['Cluster'].map(cluster_map)
  print(df.Cluster.value_counts())
  #Positive Word Cloud
  df_positive = df[df["Cluster"] == 'Positive']
  text_pos = df_positive.cleaned_head_line.str.cat()
  #Positive Word Cloud
  df_negative = df[df["Cluster"] == 'Negative']
  text_neg = df_negative.cleaned_head_line.str.cat()
  #Positive Word Cloud
  df_neutral = df[df["Cluster"] == 'Neutral']
  text_neu = df_neutral.cleaned_head_line.str.cat()

  print(df_positive.head())
  print(df_positive.shape)
  
  show_pca(df)
  show_wordcloud(text_pos)
  show_wordcloud(text_neg)
  show_wordcloud(text_neu)
  df.to_csv('MBK_Cluster_result.csv',index=False)
  return df

In [ ]:
data1.head()

In [ ]:
data1.shape

In [ ]:
Kmeans_df = Kmeans_Clustering(data1)


In [ ]:
Kmeans_df.head()

In [ ]:
data2.shape

In [ ]:
MBK_df = MBK_Clustering(data2)


In [ ]:
MBK_df.head()

In [ ]:
MBK_df.to_csv('MBK_Cluster_result.csv',index=False)

Now, we found the sentiment for our headlines with Kmeans and Mini batch Kmeans clustering methods. We got different results from two methods and the cluster forms also different. We got more balanced result from Kmeans algorithm rather than MiniBatch Kmeans.

Then  we are doing the classification with our clustered data using Naivebayes classification algorithm.

# Classification

We are using Kmeans clusters for classification.We are training Naive Bayes algorithm for predicting the sentiments. Because in Kmeans , we have more balanced data than Mini Batch Kmeans.

### EDA

####Loading Data

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from textblob import TextBlob
from nltk.probability import FreqDist
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay,classification_report,accuracy_score
from sklearn.naive_bayes import MultinomialNB
# Import label encoder
from sklearn import preprocessing
nltk.download('wordnet')
nltk.download('omw-1.4')
plt.xticks(rotation=70)
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', 100)
%matplotlib inline

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Amazon_Product_data/Kmeans_Cluster_result.csv')
data.head()

In [ ]:
data.shape

In [ ]:
new_data = data[['Description','Cluster']].copy()
new_data.rename(columns={'Cluster': 'Sentiment'}, inplace=True)

In [ ]:
new_data.head()

In [ ]:
def preprocess_text(text: str, remove_stopwords: bool) -> str:
    """This utility function sanitizes a string by:
    - removing links
    - removing special characters
    - removing numbers
    - removing stopwords
    - transforming in lowercase
    - removing excessive whitespaces
    Args:
        text (str): the input text you want to clean
        remove_stopwords (bool): whether or not to remove stopwords
    Returns:
        str: the cleaned text
    """
    lm=WordNetLemmatizer()
    # remove links
    text = re.sub(r"http\S+", "", text)
    # remove special chars and numbers
    text = re.sub("[^A-Za-z]+", " ", text)
    # remove stopwords
    if remove_stopwords:
        # 1. tokenize
        tokens = nltk.word_tokenize(text)
        # 2. check if stopword
        tokens = [w for w in tokens if not w.lower() in stopwords.words("english")]
        
        lemmed_words = [lm.lemmatize(word) for word in tokens]
        # cleaned_word_list = ' '.join(lemmed_words)
        # 3. join back together
        text = " ".join(lemmed_words)
    # return text in lower case and stripped of whitespaces
    text = text.lower().strip()
    # print(text)
    return text

In [ ]:
new_data['cleaned_description']= new_data['Description'].apply(lambda x: preprocess_text(x, remove_stopwords=True))


In [ ]:
new_data.head()

In [ ]:
new_data['text_blob_sentiment'] = new_data['cleaned_description'].apply(lambda x: TextBlob(x).sentiment.polarity)
new_data.head()

In [ ]:
plt.figure(figsize=(50,30))
plt.margins(0.02)
plt.xlabel('text_blob_sentiment', fontsize=50)
plt.xticks(fontsize=40)
plt.ylabel('Frequency', fontsize=50)
plt.yticks(fontsize=40)
plt.hist(new_data['text_blob_sentiment'], bins=50)
plt.title('text_blob_sentiment Distribution', fontsize=60)
plt.show()

In [ ]:
new_data['word_count'] = new_data['cleaned_description'].apply(lambda x: len(str(x).split()))
new_data['text_len'] = new_data['cleaned_description'].astype(str).apply(len)
new_data.head()

In [ ]:
new_data.describe()

In [ ]:
correlation = new_data[['text_blob_sentiment', 'text_len', 'word_count']].corr()
mask = np.zeros_like(correlation, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
plt.figure(figsize=(50,30))
plt.xticks(fontsize=40)
plt.yticks(fontsize=40)
sns.heatmap(correlation, cmap='coolwarm', annot=True, annot_kws={"size": 40}, linewidths=10, vmin=-1.5, mask=mask)

In [ ]:
def show_wordcloud(text):

  print(text)
  # wordcloud = WordCloud().generate(text)
  wordcloud = WordCloud(width = 2000, height = 2000,
                  background_color ='white',
                  min_font_size = 10).generate(text)

  # plot the WordCloud image                      
  plt.figure(figsize = (8, 8), facecolor = None)
  plt.imshow(wordcloud)
  plt.axis("off")
  plt.tight_layout(pad = 0)
  
  # plt.show()
  # plt.imshow(wordcloud, interpolation='bilinear')
  # plt.axis("off")

In [ ]:
words =  new_data.cleaned_description.str.cat()
print(len(words))
show_wordcloud(words)

Visualizing The Sentiment Distribution

In [ ]:
#Countplot
sns.countplot(new_data.Sentiment)

In [ ]:
words = new_data['cleaned_description'].values.tolist()
allwords = []
for doc in words:
    
    allwords.extend(doc.split())
print(allwords)
print(len(allwords))
# show_wordcloud(words)

In [ ]:
mostcommon = FreqDist(allwords).most_common(100)
wordcloud = WordCloud(width=1600, height=800, background_color='white').generate(str(mostcommon))
fig = plt.figure(figsize=(30,10), facecolor='white')
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.title('Top 100 Most Common Words', fontsize=100)
plt.tight_layout(pad=0)
plt.show()

In [ ]:
mostcommon_small = FreqDist(allwords).most_common(25)
x, y = zip(*mostcommon_small)
plt.figure(figsize=(50,30))
plt.margins(0.02)
plt.bar(x, y)
plt.xlabel('Words', fontsize=50)
plt.ylabel('Frequency of Words', fontsize=50)
plt.yticks(fontsize=40)
plt.xticks(rotation=60, fontsize=40)
plt.title('Frequency of 25 Most Common Words', fontsize=60)
plt.show()

### Prepare Train Data

In [ ]:
new_data.head()

In [ ]:
new_data.to_csv('classification_data.csv',index=False)

In [ ]:
train_data = new_data[['cleaned_description','Sentiment']].copy()
train_data.head()

###Label Encoding the Target Sentiment Column


In [ ]:

  
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
  
# Encode labels in column 'species'.
train_data['Sentiment']= label_encoder.fit_transform(train_data['Sentiment'])
  
train_data['Sentiment'].unique()
train_data["Sentiment"] = train_data["Sentiment"].astype('category')

In [ ]:
train_data.dtypes

In [ ]:

list(label_encoder.inverse_transform([0,1,2]))

In [ ]:
train_data.head()

In [ ]:
X=train_data['cleaned_description']
y = train_data['Sentiment']
X.head()

In [ ]:
type(X)

In [ ]:
type(y)

In [ ]:
vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=1, max_df=0.5)
def tf_idf(X):
  # initialize the vectorizer
  
  # fit_transform applies TF-IDF to clean texts - we save the array of vectors in X
  X = vectorizer.fit_transform(X).toarray()
  print(X.shape)
  return X

In [ ]:
X= tf_idf(X)

In [ ]:
X_train,X_test, Y_train, Y_test = train_test_split(X, y,test_size=0.10,random_state = 5,stratify = train_data['Sentiment'])

###Multinomial Naive Bayes Classifier

In [ ]:
model = MultinomialNB()
model.fit(X_train, Y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred

In [ ]:
Y_test

In [ ]:
#Actual value and the predicted value
pred_df = pd.DataFrame({'Actual value': Y_test, 'Predicted value':y_pred})
print(model.get_params(deep=True))
pred_df.head()

### Evaluation

Test Data

In [ ]:
cm = confusion_matrix(Y_test, y_pred,labels=model.classes_)
 
print ("Confusion Matrix : \n", cm)
disp = ConfusionMatrixDisplay.from_predictions(Y_test,y_pred)
# disp.plot()
plt.savefig('MNB_CM_test_data.png')
plt.show()
print(cm.ravel())
# tn, fp, fn, tp = cm.ravel()
# print(tn, fp, fn, tp)

#Accuracy
print ("Accuracy : ", accuracy_score(Y_test, y_pred))

#Precision Recall and F1-score
# logit_model=sm.Logit(y,X)
# result=logit_model.fit()
# print(result.summary2())
print(classification_report(Y_test, y_pred))

Train Data

In [ ]:
y_pred = model.predict(X_train)

In [ ]:
#Actual value and the predicted value
pred_df = pd.DataFrame({'Actual value': Y_train, 'Predicted value':y_pred})
print(model.get_params(deep=True))
pred_df.head()

In [ ]:
cm = confusion_matrix(Y_train, y_pred,labels=model.classes_)
 
print ("Confusion Matrix : \n", cm)
disp = ConfusionMatrixDisplay.from_predictions(Y_train,y_pred)
# disp.plot()
plt.savefig('MNB_CM_Train_data.png')
plt.show()
print(cm.ravel())
# tn, fp, fn, tp = cm.ravel()
# print(tn, fp, fn, tp)

#Accuracy
print ("Accuracy : ", accuracy_score(Y_train, y_pred))

#Precision Recall and F1-score
# logit_model=sm.Logit(y,X)
# result=logit_model.fit()
# print(result.summary2())
print(classification_report(Y_train, y_pred))

#Conclusion

Here, we tried first clustering and then classification for sentiment analysis. But the results from both methods are not satisafctory due to the less amount of records. We have  done exploratory data analysis with clustered data and then applied naive bayes algorithm
